In [1]:
import keras.utils
import numpy as np
import tqdm.notebook
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten

inp = Input(shape=(240, 320, 1))

n_filters = 512
max_p = inp

for _ in range(3):
    conv = Conv2D(filters=n_filters, kernel_size=5, activation='relu')(max_p)
    max_p = MaxPooling2D(pool_size=(2, 2))(conv)
    
    n_filters //= 2

conv = Conv2D(filters=n_filters, kernel_size=3, activation='relu')(max_p)
max_p = MaxPooling2D(pool_size=(2, 2))(conv)

flatten = Flatten()(max_p)
dense = Dense(128, activation='relu')(flatten)
# out = Dense(14, activation='relu')(dense)

head_out = Dense(1, activation='relu', name='head_out')(dense)


model = Model(
    inputs=inp,
    outputs=[
    head_out
    ],
    name='head_only'
)

#model.save_weights("default.weights.h5")

#model.summary()

In [2]:
import os
import cv2
import random
from tqdm import tqdm_notebook

class Database_Loader(keras.utils.Sequence):
    
    def __init__(self, image_location, data_location, sample_count, batch_size, shuffle=True, seed=0, input_dimensions=(240, 320), prefix="Avatar_", use_memory=False, load_data = True, random_sample = False):
        self.image_location = image_location
        self.data_location = data_location
        self.sample_count = sample_count
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.seed = seed
        self.input_dimensions = input_dimensions
        self.prefix = prefix
        self.use_memory = use_memory
        self.load_data = load_data
        self.random_sample = random_sample

        if self.random_sample:
            self.IDs = random.sample(range(len(os.listdir(self.image_location))),self.sample_count)
        else:
            self.IDs = [x for x in range(self.sample_count)]
            self._load_data()

        super().__init__(workers=2, use_multiprocessing=True)
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(self.sample_count / self.batch_size))
    
    
    def _load_data(self):
        if self.load_data:
            self.data = dict()
            for index in self.IDs:
                self.data[index] = np.load(os.path.join(self.data_location + self.prefix + f"{index:06d}.npy"))

        if self.use_memory:
            self.images = dict()
            for index in self.IDs:
                self.images[index] = cv2.imread(os.path.join(self.image_location + self.prefix + f"{index:06d}.png"), cv2.IMREAD_GRAYSCALE)
    
    def __getitem__(self, index):
        X = np.empty(shape=(self.batch_size, 240, 320))
        y = {
            'head_out': [],
        }
        
        start_index = index * self.batch_size + 1
        
        for i in range(self.batch_size):
            name = f"{self.IDs[(start_index + i)%self.sample_count]:06d}"
            
            if self.load_data:
                current_measurement = self.data[self.IDs[(start_index + i)%self.sample_count]][:-1]          
            else:
                current_measurement = np.load(os.path.join(self.data_location + self.prefix + name + ".npy"))[:-1]
                
            if self.use_memory:
                X[i,] = self.images[self.IDs[(start_index + i)%self.sample_count]]
                
            else:
                X[i,] = cv2.imread(os.path.join(self.image_location + self.prefix + name + ".png"), cv2.IMREAD_GRAYSCALE)

            y['head_out'].append([current_measurement[0]])

        for key, value in y.items():
            y[key] = np.array(value)
            
        return X, y
    
    def on_epoch_end(self):
        if self.random_sample:
            self.IDs = random.sample(range(len(os.listdir(self.image_location))),self.sample_count)
            self._load_data()
        
        if self.shuffle:
            np.random.seed(self.seed)
            np.random.shuffle(self.IDs)
        else:
            self.IDs = np.arange(self.sample_count)
            
    
    
        

In [3]:
# quickTrain = {'image_location': "../Surreact-APose/train/imgs_nobg_frontEDITED/", 
#               'data_location': "../Surreact-APose/train/measurements/", 
#               'sample_count': 5000, 
#               'batch_size': 4, 
#               'seed': 69, #np.random.randint(0, 10000), 
#               'use_memory': False,
#               'random_sample': True}
# 
# quickValidate = {'image_location': "../Surreact-APose/train/imgs_nobg_frontEDITED/",
#               'data_location': "../Surreact-APose/train/measurements/",
#               'sample_count': 1000,
#               'batch_size': 4,
#               'seed': 69, #np.random.randint(0, 10000),
#               'use_memory': False,
#               'random_sample': True}
# 
# train_generator = Database_Loader(**quickTrain)
# validation_generator = Database_Loader(**quickValidate)


import cv2
import os
import numpy as np
import tqdm

X = []
y_true = []

for i in tqdm.notebook.tqdm(range(1000)):
    X.append(cv2.imread(os.path.join(f"../Surreact-APose/train/imgs_nobg_frontEDITED/Avatar_000000.png"), cv2.IMREAD_GRAYSCALE))
    y_true.append(np.load(os.path.join(f"../Surreact-APose/train/measurements/Avatar_000000.npy"))[0])


X = np.array(X)
y_true = np.array(y_true)

# X = np.array([data[0] for data in dataset.values()])
# y_true = np.array([data[1] for data in dataset.values()])
# 
# (
#     y_true_head, y_true_neck, y_true_sts,y_true_arm, y_true_stw, y_true_torso, y_true_bicep, y_true_wrist, y_true_chest, y_true_waist, y_true_pelvis, y_true_leg, y_true_inner_leg, y_true_thigh, y_true_knee, y_true_calf, height
# ) = np.hsplit(y_true, 17) #17 is height


  0%|          | 0/1000 [00:00<?, ?it/s]

In [4]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.losses import MeanSquaredError
from keras_tqdm import TQDMNotebookCallback

model.load_weights("default.weights.h5")

model.compile(
    optimizer=Adam(),
    loss='mse',
)
keras.utils.set_random_seed(42)


checkpoint_filepath = './models/checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True)


model.fit(
    x=X,
    y=y_true,
    validation_split=0.2,
    callbacks=[
        
        TensorBoard(write_graph=False,log_dir="./logs"),
        # TQDMNotebookCallback(separator="", metric_format="{name}: {value:0.8f} || ")
        # model_checkpoint_callback
    ],
    batch_size=1,
    epochs=200,
    verbose=1,
    shuffle=False
)

# 
# model.fit(
#     x=X,
#     y={
#         'head_out': y_true_head,
#         'neck_out': y_true_neck,
#         'shoulder_to_shoulder_out': y_true_sts,
#         'arm_out': y_true_arm,
#         'shoulder_to_wrist_out': y_true_stw,
#         'torso_out': y_true_torso,
#         'bicep_out': y_true_bicep,
#         'wrist_out': y_true_wrist,
#         'chest_out': y_true_chest,
#         'waist_out': y_true_waist,
#         'pelvis_out': y_true_pelvis,
#         'leg_out': y_true_leg,
#         'inner_leg_out': y_true_inner_leg,
#         'thigh_out': y_true_thigh,
#         'knee_out': y_true_knee,
#         'calf_out': y_true_calf
#     },
#     batch_size=8,
#     epochs=10,
#     validation_data=train_data[1],
#     shuffle=True,
#     callbacks=[TensorBoard(write_graph=False)]
# )

2024-03-31 14:55:56.316271: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-31 14:55:57.410937: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/200


2024-03-31 14:55:57.964691: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-03-31 14:55:57.964896: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3352 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


366/800 ━━━━━━━━━━━━━━━━━━━━ 25s 60ms/step - loss: 0.8881

KeyboardInterrupt: 

In [ ]:
    #Data modifiaction

# import os
# from tqdm import tqdm
# 
# avatar_count = 79999
# counter = 0
# 
# for number in tqdm(range(avatar_count)):
#     if number == 5:
#         continue
#     file = (tuple(np.load(f"../Surreact-APose/train/measurements/Avatar_{number:06d}.npy", allow_pickle=True)))
#     measurements = []
#     for value in file:
#         measurements.append(float(value))
#     np.save(f"../Surreact-APose/train/measurements/Avatar_{counter:06d}", measurements)
#     counter += 1

In [ ]:
#Image modification

# import os
# import cv2
# from tqdm import tqdm
# 
# 
# avatar_count = 79999
# counter = 0
# 
# for number in tqdm(range(avatar_count)):
#     if number == 5:
#         continue
#     file = cv2.imread(f"../Surreact-APose/train/imgs_nobg_front/Avatar_{number:06d}.png", cv2.IMREAD_UNCHANGED)
#     file = cv2.cvtColor(file, cv2.COLOR_mRGBA2RGBA)
#     ret, binary = cv2.threshold(file, 0, 255, 0)
# 
#     cv2.imwrite(f"../Surreact-APose/train/imgs_nobg_frontEDITED/Avatar_{counter:06d}.png", binary)
#     counter +=1

In [ ]:
# from tensorboard.backend.event_processing import event_accumulator
# import pandas as pd
# 
# 
# def parse_tensorboard(path, scalars):
#     """returns a dictionary of pandas dataframes for each requested scalar"""
#     ea = event_accumulator.EventAccumulator(
#         path,
#         size_guidance={event_accumulator.SCALARS: 0},
#     )
#     _absorb_print = ea.Reload()
#     print(ea.Tags()["tensors"])
#     # make sure the scalars are in the event accumulator tags
#     assert all(
#         s in ea.Tags()["tensors"] for s in scalars
#     ), "some scalars were not found in the event accumulator"
#     return {k: pd.DataFrame(ea.Tensors(k)) for k in scalars}
# 
# 
# 
# a = parse_tensorboard("logsTest/train/events.out.tfevents.1711732161.root.21184.39.v2",['epoch_head_out_mae', 'batch_head_out_mae'])
# 
# 
# print(a)